# 1. Learn Boolean decision rules
### Import libraries

In [1]:
from rulelearning.imli import imli as imli
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

### Model Configuration

We now create an instance of `imli` object. First we learn a classification rule in CNF, that is, the decision rule is ANDs of ORs of input features. For that, we specify `rule_type=CNF` inside the model. In this example, we learn a 2 clause rule with parameter `data_fidelity=10`. `data_fidelity` parameter sets the priority between accuracy and rule-sparsity such that a higher value of `data_fidelity` results in a more accurate rule. We require a MaxSAT solver to learn the Boolean rule. In this example, we use `open-wbo` as the MaxSAT solver. 

In [2]:
model = imli(rule_type="CNF", num_clause=2,  data_fidelity=10, solver="open-wbo", work_dir="rulelearning/", verbose=True)

### Load dataset
In this example, we learn a decision rule on `Iris` flower dataset. While the original dataset is used for multiclass classification, we modify it for binary classification. Our objective is to learn a decision rule that separates `Iris Versicolour` from other two classes of Iris: `Iris Setosa` and `Iris Virginica`. 

Our framework requires the training set to be discretized. In the following, we apply entropy-based discretization on the dataset. We have also implemented quantile-based discretization (call `X, y, features = model.discretize("benchmarks/iris.csv")`).

In [3]:
X, y, features = model.discretize_orange("benchmarks/iris_orange.csv")

Applying entropy based discretization using Orange library
- file name:  benchmarks/iris_orange.csv
- the number of discretized features: 22


### Split dataset into train and test set

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Train the model

In [5]:
model.fit(X_train,y_train)


Training started for batch:  1
- number of soft clauses:  93
- number of Boolean variables: 157
- number of hard and soft clauses: 863


Batch tarining complete
- number of literals in the rule: 2
- number of training errors:    3 out of 49

Training started for batch:  2
- number of soft clauses:  95
- number of Boolean variables: 161
- number of hard and soft clauses: 890


Batch tarining complete
- number of literals in the rule: 4
- number of training errors:    1 out of 51


### The following function is used to access the performance of the trained model

In [6]:
def measurement(cnf_matrix):
    # print(cnf_matrix)
    TN, FP, FN, TP = cnf_matrix.ravel()

    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
    # Negative predictive value
    NPV = TN/(TN+FN)
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
    # False negative rate
    FNR = FN/(TP+FN)
    # False discovery rate
    FDR = FP/(TP+FP)

    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
    return TPR, TNR, PPV, NPV, FPR, FNR, FDR, ACC*100

### Report performance of the learned rule

In [7]:
yhat_train = model.predict(X_train, y_train)
_, _, _, _, _, _, _, train_acc = measurement(confusion_matrix(y_train, yhat_train))
yhat_test = model.predict(X_test, y_test)
_, _, _, _, _, _, _, test_acc = measurement(confusion_matrix(y_test, yhat_test))
print("\ntraining    accuracy: ", train_acc)
print("test        accuracy: ", test_acc)



Prediction through MaxSAT formulation
- number of soft clauses:  188
- number of Boolean variables: 274
- number of hard and soft clauses: 1753

Prediction through MaxSAT formulation
- number of soft clauses:  138
- number of Boolean variables: 164
- number of hard and soft clauses: 973

training    accuracy:  96.0
test        accuracy:  98.0


### Show the learned rule

In [8]:
rule = model.get_rule(features)
print("Learned rule is: ")
print(rule)

Learned rule is: 
( sepal length = (5.45 - 7.05) OR petal length = (2.45 - 4.75) ) AND 
( petal length = (2.45 - 4.75) OR petal width = (0.8 - 1.75))


# 2. Learn decision rules as DNF

To learn a decision rule as a DNF (ORs of ANDs of input features), we specify `rule_type=DNF` in the parameters of the model. In the following, we learn a 2 clause DNF decision rule. 

In [9]:
model = imli(rule_type="DNF", num_clause=2,  data_fidelity=10, solver="open-wbo", work_dir="rulelearning/", verbose=False)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X, y, features = model.discretize_orange("benchmarks/iris_orange.csv")
model.fit(X_train,y_train)
yhat_train = model.predict(X_train, y_train)
_, _, _, _, _, _, _, train_acc = measurement(confusion_matrix(y_train, yhat_train))
yhat_test = model.predict(X_test, y_test)
_, _, _, _, _, _, _, test_acc = measurement(confusion_matrix(y_test, yhat_test))
print("\ntraining    accuracy: ", train_acc)
print("test        accuracy: ", test_acc)
rule = model.get_rule(features)
print("\nLearned rule is: ")
print(rule)


training    accuracy:  96.0
test        accuracy:  98.0

Learned rule is: 
( sepal length >=  7.05 AND not_petal width = (0.8 - 1.75) ) OR 
( not_petal length = (2.45 - 4.75))


# 3. Learn more expressible decision rules

Our framework allows one to learn more expressible decision rules, which we call relaxed_CNF rules. This rule allows thresholds on satisfaction of clauses and literals and can learn more complex decision boundaries. See the [ECAI-2020](https://bishwamittra.github.io/publication/ecai_2020/paper.pdf) paper for more details. 


In our framework, set the parameter `rule_type=relaxed_CNF` to learn the rule.

In [11]:
model = imli(rule_type="relaxed_CNF", num_clause=2,  data_fidelity=10, solver="cplex", work_dir="rulelearning/", verbose=False)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X, y, features = model.discretize_orange("benchmarks/iris_orange.csv")
model.fit(X_train,y_train)
yhat_train = model.predict(X_train, y_train)
_, _, _, _, _, _, _, train_acc = measurement(confusion_matrix(y_train, yhat_train))
yhat_test = model.predict(X_test, y_test)
_, _, _, _, _, _, _, test_acc = measurement(confusion_matrix(y_test, yhat_test))
print("\ntraining    accuracy: ", train_acc)
print("test        accuracy: ", test_acc)


training    accuracy:  95.0
test        accuracy:  98.0


### Understanding the decision rule

In this example, we ask the framework to learn a 2 clause rule. During training, we learn the thresholds on clauses and literals while fitting the dataset. The learned rule operates in two levels. In the first level, a clause is satisfied if the literals in the clause satisfy the learned threshold on literals. In the second level, the formula is satisfied when the threshold on clauses is satisfied.

In [14]:
rule = model.get_rule(features)
print("Learned rule is: ")
print(rule)
print("\nThrehosld on clause:", model.get_threshold_clause())
print("Threshold on literals: (this is a list where the entrie denotes threholds on literals on all clauses)")
print(model.get_threshold_literal())

Learned rule is: 
[ (  petal length = (2.45 - 4.75)  + petal width = (0.8 - 1.75)   )>= 1  ] +
[ ( )>= 0  ]  >= 2

Threhosld on clause: 2
Threshold on literals: (this is a list where the entrie denotes threholds on literals on all clauses)
[1, 0]
